In [61]:
import pandas as pd 

algorithm = "ElasticNet"
algorithm = "LightGBM"

scores = pd.read_csv(
    f"../../data/page7_MultivariateXWASResults/Scores/Scores_{algorithm}_test.csv",
    index_col=0,
)
scores.rename(columns={"env_dataset": "category", "organ": "dimension"}, inplace=True)
scores

,category,dimension,std,r2,sample_size,subset
0,Clusters_HandGripStrength,\*,0.013150,0.069580,80343,Biomarkers
1,Clusters_HandGripStrength,*instances01,0.034026,0.057859,4404,Biomarkers
2,Clusters_HandGripStrength,*instances1.5x,0.018984,0.069503,73983,Biomarkers
3,Clusters_HandGripStrength,*instances23,0.035655,0.016081,1956,Biomarkers
4,Clusters_HandGripStrength,Abdomen,0.005387,0.021373,36654,Biomarkers
...,...,...,...,...,...,...
2272,medical_diagnoses_Z,MusculoskeletalHips,0.015131,0.024697,42146,Pathologies
2273,medical_diagnoses_Z,MusculoskeletalKnees,0.009602,0.023231,39809,Pathologies
2274,medical_diagnoses_Z,MusculoskeletalFullBody,0.008037,0.030279,42164,Pathologies
2275,medical_diagnoses_Z,MusculoskeletalScalars,0.004268,0.067414,486642,Pathologies


In [ ]:
DICT_TO_CHANGE_DIMENSIONS = {"ImmuneSystem": "BloodCells", "\*" : "set", "*instances01": "set_instances01", "*instances1.5x", "*instances23": "set_instances23"}
DICT_TO_CHANGE_CATEGORIES = {
    "HeartSize": "HeartFunction",
    "AnthropometryImpedance": "Impedance",
    "AnthropometryBodySize": "Anthropometry",
    "Claudification": "Claudication"
}
DICT_TO_CHANGE_MAIN_CATEGORIES = {"Pathologies": "Diseases"}



In [58]:
print(scores.loc["Pathologies", "category"].drop_duplicates().shape[0])
for cat in scores.loc["Pathologies", "category"].drop_duplicates():
    if cat not in MAIN_CATEGORIES_TO_CATEGORIES["Diseases"]:
        print(cat)

26


In [51]:
from dash_website import MAIN_CATEGORIES_TO_CATEGORIES

MAIN_CATEGORIES_TO_CATEGORIES.keys()

dict_keys(['All', 'Biomarkers', 'ClinicalPhenotypes', 'Diseases', 'Environmental', 'Socioeconomics'])